# Final Project

Tony Nguyen

CPSC 222 01

Dr. Gina Sprint

December 13th, 2022

## Introduction

For our class final project, I choose to work primarily with my **Apple Health Sleep Data** and **Netflix Watching History**.

Throughout this project, I want to learn more about my sleep history, specifically, how long would I sleep each night on average. I realize that it is instrumental to have a decent amount of sleep since we are all working in a high academic-demadning environment; thus, being able to sleep well is one of the easiest way to prevent ourselves from burning out.

I am a big Netflix user. Watching series on Netflix is one of my ways to relax after school and work. Therefore, I want to know if there would be a correlation between the amount of serie episodes or movies I watched and the total time I got to sleep each night.

I hope the result of this project can provide a glimpse of my sleep routine, something that I usually take for granted without thinking much about how much I had actually slept the night before. At the same time, I want to know if Netflix would play a role in my sleep routine, since I find myself binge watch a lot. Besides, as the Apple Health Data is very extensive that covers other data type, hopefully in the future I can do some further analysis with other health data and find something interesting like this one.

**TODO: STAKEHOLDERS INTERESTED IN THE RESULT**

### Apple Health Sleep Dataset

The original Apple Health Dataset takeout contains several tables in different formats including electrocardiogram in CSV, workout routes in GPX, health clinical reports in JSON and an XML that has other data points that my phone and my watch collects. The file type I will be working with is XML.

In this original XML file called `export.xml`, there are multiple entries for different type of health information, including the sleep data that I need. And since this XML file is too big, at approximately 390MB, it takes my computer a very long time to work with if I load it directly into a DataFrame. It is essentially unwsie since I will need to re-run this notebook multiple times during my coding process.

Therefore, in the file `healthdata_preprocessing.py`, I load the original XML file into a DataFrame, then export it to a CSV file called `export_converted.csv`. Although the exported CSV size is still relatively big at roughly 260MB, the time it takes to run is significantly faster.

***Note**: In order for the `pandas.read_csv()` function to work, I manually remove the first 213 lines (out of 1,443,267 lines) in the XML file. Those 213 lines are Apple's description of how to interprete the instances.*

From this point forward, I will use the CSV file to work with the health data. After the format conversion, there is a total of 946,979 instances, varies across different data type. The attributes of this dataset is as follow:
* `type`: Type of data point. I use this one to filter sleep data later.
* `sourceName`: Where does that data come from. The common instances are my iPhone and my Apple Watch. There are also other instances such as the Health app or *MyChart*, the portal that my hospital use.
* `sourceVersion`: The software version of `sourceName`.
* `unit`: The unit of data records. Depending on which type of data it is, there will be a corresponding unit.
* `creationDate`: The time when an instance is logged.
* `startDate`: For those entries that record a period, this is the time when the record starts. 
* `endDate`: For those entries that record a period, this is the time when the record ends. 
* `value`: The value of a record. E.g., duration of sleep for sleep data
* `device`: Hardware information, if applicable. It contains `sourceName`'s name, manufacturer, model, hardware version, and software version.
* `MetadataEntry`: Include a key-value pair.
* `HeartRateVariabilityMetadataList`: For heart rate data, record a list of Instantaneous Beat Per Minute.

Apple has a specific sleep mode that I have been using to record my sleep information. Every night just before I put my phone down and go to sleep, I turn sleep mode on, which prompts the phone to start counting my sleep time. And the next morning, at the wake up time that I set earlier, it will start playing alarm sound from quieter to lounder so as to not aruptly wake me up, which can deliver a better sleep experience. If I happen to wake up during the night and go back to sleep later, it can automaticaly subtract the actual time that I am awake from my sleep time. The whole process can get even more precise if paired with an Apple Watch, which can analyze different sleep stages and respiratory rate. These information can be really helpful for further analysis, but since most of the time I do not wear my watch to sleep, I decide to not include those sleep stages information as they are insufficient on a daily basi. Read more about Apple's sleep mode [here](https://support.apple.com/en-us/HT211685).

**TODO: Classification Task**
